In [1]:
import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [4]:
df_google = pd.read_csv('../data/mined/GOOGLE1.csv')

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


In [3]:
embeddings = elmo(
["the cat is on the mat", "dogs are in the fog"],
signature="default",
as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [4]:
sess = tf.Session()

In [5]:
sess.run(tf.global_variables_initializer())

In [6]:
sess.run(embeddings)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node module_apply_default/bilm/CNN_1/Conv2D_2 (defined at /home/vincent/anaconda2/envs/cds/lib/python3.6/site-packages/tensorflow_hub/native_module.py:447)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](module_apply_default/bilm/CNN_1/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, module_apply_default/bilm/CNN_1/Conv2D_2/ReadVariableOp/_81)]]
	 [[{{node module_apply_default/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/stack/_231}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1066_module_apply_default/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/

In [8]:
tokens_input = [["the", "cat", "is", "on", "the", "mat"],
["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]
embeddings = elmo(
inputs={
"tokens": tokens_input,
"sequence_len": tokens_length
},
signature="tokens",
as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
